<a href="https://colab.research.google.com/github/StratagemGIS/notebooks/blob/main/data_processing/82_busiest_bus_stops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vaasudevan Srinivasan 🧑🏻‍💻  
StratagemGIS Solutions

In [ ]:
%%capture
!pip install mapclassify

In [ ]:
import geopandas as gpd
import pandas as pd
import pooch

In [ ]:
bus_stops_file = pooch.retrieve(
    'https://github.com/StratagemGIS/datasets/raw/refs/heads/main/vector/FrederictonTransit_WeekdayBusStops.geojson',
    known_hash='d5b2abdb81249f02bea4c274785664519b55c30c766b88f9d2d9fa15fc0944a4'
)

ridership_file = pooch.retrieve(
    'https://github.com/StratagemGIS/datasets/raw/refs/heads/main/vector/12_September_2024_Ridership.csv.zip',
    known_hash='a0efe5d16fc356b337592ff55348564ba1c983f14695868ea3da593cb4e5ce1e'
)

In [ ]:
bus_stops = (
    gpd.read_file(bus_stops_file)
    .dropna(subset=['RouteName', 'stop_name'])
    .drop_duplicates(subset=['geometry'])
    .assign(Route=lambda df_: df_.RouteName.str.split().str[0])
    .rename(columns={'stop_name': 'Location'})
    [['Route', 'Location', 'geometry']]
)

ridership = (
    pd.read_csv(ridership_file, low_memory=False)
    .groupby(['Route', 'Stop'], as_index=False)
    .agg({'Ons': 'sum', 'Offs': 'sum'})
    .assign(Count=lambda df_: df_.Ons + df_.Offs)
    .rename(columns={'Stop': 'Location'})
    [['Route', 'Location', 'Count']]
)

merged = (
    bus_stops.merge(ridership, on=['Route', 'Location'])
    .sort_values('Count', ascending=False)
    .assign(radius=lambda df_: df_.Count / df_.Count.max() * 15)
    .reset_index(drop=True)
)

In [ ]:
merged.head(5)

,Route,Location,geometry,Count,radius
0,17S,Regent Mall,POINT (-66.66288 45.93526),3248,15.000000
1,11S,Windsor and/et Kings College,POINT (-66.64478 45.94772),2749,12.695505
2,17S,UNB SUB,POINT (-66.64226 45.94524),2631,12.150554
3,11S,Windsor and/et Head Hall,POINT (-66.64331 45.94958),2429,11.217672
4,11S,Regent and/et Prospect,POINT (-66.65729 45.93982),2227,10.284791


In [10]:
m = merged.explore(
    column='Count', cmap='Spectral_r',
    tiles='CartoDB dark_matter', legend_kwds={'colorbar': False},
    style_kwds={'style_function':lambda x: {'radius': x['properties']['radius']}},
    # width=700, height=600,
)
m

In [11]:
m.save('82_busiest_bus_stops.html')